In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:

# Dataset
tanaman_hama = pd.DataFrame({
    'Tanaman': ['Cabe', 'Cabe', 'Pare', 'Pare'],
    'Jenis Hama': ['Kutu Daun', 'Ulat', 'Ulat', 'Belalang']
})

# Dataset Pestisida dan Jenis Hama yang Dapat Dibasmi
pestisida_hama = pd.DataFrame({
    'Pestisida': ['Pestisida A', 'Pestisida B', 'Pestisida C'],
    'Jenis Hama yang Dibasmi': [['Kutu Daun'], ['Ulat'], ['Ulat', 'Belalang']]
})

# Ekspansi baris untuk setiap elemen dalam list
pestisida_hama_expanded = pestisida_hama.explode('Jenis Hama yang Dibasmi')

# Gabungkan dataset berdasarkan jenis hama
dataset = pd.merge(tanaman_hama, pestisida_hama_expanded, left_on='Jenis Hama', right_on='Jenis Hama yang Dibasmi', how='left')

display(dataset)


,Tanaman,Jenis Hama,Pestisida,Jenis Hama yang Dibasmi
0,Cabe,Kutu Daun,Pestisida A,Kutu Daun
1,Cabe,Ulat,Pestisida B,Ulat
2,Cabe,Ulat,Pestisida C,Ulat
3,Pare,Ulat,Pestisida B,Ulat
4,Pare,Ulat,Pestisida C,Ulat
5,Pare,Belalang,Pestisida C,Belalang


In [7]:
# Pisahkan fitur dan label
X = dataset[['Tanaman', 'Jenis Hama']]
y = dataset['Pestisida']

# Encoding kategorikal data
X = pd.get_dummies(X)

# Pisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan latih model Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Lakukan prediksi
y_pred = model.predict(X_test)

# Evaluasi kinerja model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Contoh prediksi
new_data = pd.DataFrame({'Tanaman': ['Cabe', 'Pare', 'Cabe'],
                         'Jenis Hama': ['Kutu Daun', 'Belalang', 'Ulat']})
new_data_encoded = pd.get_dummies(new_data)
predictions = model.predict(new_data_encoded)
print("Rekomendasi Pestisida untuk Setiap Tanaman:", predictions)

Accuracy: 0.0
Rekomendasi Pestisida untuk Setiap Tanaman: ['Pestisida C' 'Pestisida C' 'Pestisida C']


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Data tanaman dan hama
tanaman_hama_data = {
    'tanaman': ['Padi', 'Jagung', 'Tomat', 'Kentang'],
    'hama': [['Wereng', 'Ulat Grayak'], ['Ulat Grayak', 'Kutu Daun'], ['Bakteri X'], ['Kutu Daun']]
}

# Data pestisida dan jenis hama yang dibasmi
pestisida_data = {
    'pestisida': ['PestA', 'PestB', 'PestC', 'PestD', 'PestE'],
    'hama_dibasmi': [['Wereng'], ['Ulat Grayak'], ['Bakteri X'], ['Kutu Daun'], ['Ulat Grayak', 'Kutu Daun']]
}

# Membuat dataframe untuk tanaman dan hama
tanaman_hama_df = pd.DataFrame(tanaman_hama_data)

# Membuat dataframe untuk pestisida
pestisida_df = pd.DataFrame(pestisida_data)

# Fungsi untuk mencocokkan hama dengan pestisida
def match_pesticide(hama):
    for idx, row in pestisida_df.iterrows():
        if set(hama).issubset(set(row['hama_dibasmi'])):
            return row['pestisida']
    return None

# Membangun vektor fitur menggunakan TfidfVectorizer
tfidf = TfidfVectorizer()
tanaman_hama_tfidf_matrix = tfidf.fit_transform([' '.join(hama) for hama in tanaman_hama_df['hama']])

# Fungsi untuk merekomendasikan pestisida
def recommend_pesticide(tanaman1_input, tanaman2_input):
    # Menggabungkan hama dari kedua tanaman
    hama_tanaman1 = tanaman_hama_df[tanaman_hama_df['tanaman'] == tanaman1_input]['hama'].iloc[0]
    hama_tanaman2 = tanaman_hama_df[tanaman_hama_df['tanaman'] == tanaman2_input]['hama'].iloc[0]
    combined_hama = list(set(hama_tanaman1 + hama_tanaman2))
    
    # Menghitung vektor fitur untuk kombinasi hama
    input_tfidf_matrix = tfidf.transform([' '.join(combined_hama)])
    
    # Menghitung similarity antara input hama dan data pestisida
    similarity_scores = cosine_similarity(input_tfidf_matrix, tanaman_hama_tfidf_matrix)
    
    # Mendapatkan indeks pestisida dengan similarity tertinggi
    top_index = similarity_scores.argsort()[0][-1]
    
    # Mengembalikan nama pestisida yang direkomendasikan
    return pestisida_df.iloc[top_index]['pestisida']

# Meminta input dari pengguna
tanaman1_input = input("Masukkan nama tanaman pertama: ")
tanaman2_input = input("Masukkan nama tanaman kedua: ")

# Menghasilkan rekomendasi pestisida
recommended_pesticide = recommend_pesticide(tanaman1_input, tanaman2_input)
print(f"Untuk menanam {tanaman1_input} dan {tanaman2_input}, Anda dapat menggunakan pestisida {recommended_pesticide}.")


Untuk menanam Padi dan Tomat, Anda dapat menggunakan pestisida Tidak ada pestisida yang cocok ditemukan..
